## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

#reading key from higher level folder
import sys
sys.path.insert(1, '../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,26.76,96,100,0.87,snow
1,1,Rikitea,PF,-23.1203,-134.9692,75.88,80,63,17.09,light rain
2,2,Barrow,US,71.2906,-156.7887,-5.78,77,90,9.22,haze
3,3,Aksu,CN,41.1231,80.2644,36.27,51,12,4.70,few clouds
4,4,The Valley,AI,18.2170,-63.0578,78.66,72,1,13.42,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,75.88,80,63,17.09,light rain
4,4,The Valley,AI,18.2170,-63.0578,78.66,72,1,13.42,clear sky
5,5,Richards Bay,ZA,-28.7830,32.0377,77.34,55,100,19.04,overcast clouds
11,11,Acapulco,MX,16.8634,-99.8901,76.41,79,67,4.52,broken clouds
19,19,Tuy Hoa,VN,13.0833,109.3000,76.87,93,100,9.48,overcast clouds
20,20,Eyl,SO,7.9803,49.8164,86.59,56,28,15.84,scattered clouds
26,26,Vaini,TO,-21.2000,-175.2000,80.76,83,75,11.50,light rain
27,27,Ippy,CF,6.2679,21.2247,80.64,31,100,3.24,overcast clouds
29,29,Busselton,AU,-33.6500,115.3333,82.08,40,100,13.38,overcast clouds
31,31,Saint-Philippe,RE,-21.3585,55.7679,83.75,84,96,5.23,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,75.88,80,63,17.09,light rain
4,4,The Valley,AI,18.2170,-63.0578,78.66,72,1,13.42,clear sky
5,5,Richards Bay,ZA,-28.7830,32.0377,77.34,55,100,19.04,overcast clouds
11,11,Acapulco,MX,16.8634,-99.8901,76.41,79,67,4.52,broken clouds
19,19,Tuy Hoa,VN,13.0833,109.3000,76.87,93,100,9.48,overcast clouds
20,20,Eyl,SO,7.9803,49.8164,86.59,56,28,15.84,scattered clouds
26,26,Vaini,TO,-21.2000,-175.2000,80.76,83,75,11.50,light rain
27,27,Ippy,CF,6.2679,21.2247,80.64,31,100,3.24,overcast clouds
29,29,Busselton,AU,-33.6500,115.3333,82.08,40,100,13.38,overcast clouds
31,31,Saint-Philippe,RE,-21.3585,55.7679,83.75,84,96,5.23,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,75.88,light rain,-23.1203,-134.9692,
4,The Valley,AI,78.66,clear sky,18.2170,-63.0578,
5,Richards Bay,ZA,77.34,overcast clouds,-28.7830,32.0377,
11,Acapulco,MX,76.41,broken clouds,16.8634,-99.8901,
19,Tuy Hoa,VN,76.87,overcast clouds,13.0833,109.3000,
20,Eyl,SO,86.59,scattered clouds,7.9803,49.8164,
26,Vaini,TO,80.76,light rain,-21.2000,-175.2000,
27,Ippy,CF,80.64,overcast clouds,6.2679,21.2247,
29,Busselton,AU,82.08,overcast clouds,-33.6500,115.3333,
31,Saint-Philippe,RE,83.75,light rain,-21.3585,55.7679,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# replacing empty strings with NaN values
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.isnull().sum()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,75.88,light rain,-23.1203,-134.9692,People ThankYou
4,The Valley,AI,78.66,clear sky,18.2170,-63.0578,CeBlue Villas & Beach Resort
5,Richards Bay,ZA,77.34,overcast clouds,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
11,Acapulco,MX,76.41,broken clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
19,Tuy Hoa,VN,76.87,overcast clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
20,Eyl,SO,86.59,scattered clouds,7.9803,49.8164,Amiin Hotel
26,Vaini,TO,80.76,light rain,-21.2000,-175.2000,Keleti Beach Resort
29,Busselton,AU,82.08,overcast clouds,-33.6500,115.3333,Observatory Guest House
31,Saint-Philippe,RE,83.75,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
39,Dingle,PH,83.93,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID", encoding="utf_8_sig", float_format='%.2f')

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel name</dt><dd>{Hotel Name}</dd>
<dt>City name</dt><dd>{City}</dd>
<dt>Country code</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))